# Identify rhythmic genes of datasets without time annotation 

### Tools: CYCLOPS
CYCLOPS reveals human transcriptional rhythms Ron C. Anafi, Lauren J. Francey, John B. Hogenesch, Junhyong Kim Proceedings of the National Academy of Sciences May 2017, 114 (20) 5312-5317; DOI: 10.1073/pnas.1619320114

### Dataset: GTEx Lung (version 8)

### Seed genes: Baboon genes cyclying in more than 20 tissues
Diurnal transcriptome atlas of a primate across major neural and peripheral tissues
BY LUDOVIC S. MURE, HIEP D. LE, GIORGIA BENEGIAMO, MAX W. CHANG, LUIS RIOS, NGALLA JILLANI, MAINA NGOTHO, THOMAS KARIUKI, OURIA DKHISSI-BENYAHYA, HOWARD M. COOPER, SATCHIDANANDA PANDA, SCIENCE16 MAR 2018

## Import packages and modules used in CYCLOPS
Note: Change in addprocs() does not affect the number of procosser used because the function in module scripts are written for 5 processors according to Dr. Anafi's machine. If would like to change the number of processor used in CYCLOPS, edit the module script as well.

In [1]:
# Add processor
addprocs(5)

# Set working directory for each processor
@everywhere basedir = homedir()
@everywhere workdir = string(basedir, "/circa/CYCLOPS/baboon")
@everywhere cd(workdir)

using StatsBase
using MultivariateStats
using Distributions

@everywhere include("CYCLOPS_v6_2a_AutoEncoderModule_multi.jl")
@everywhere include("CYCLOPS_v6_2a_Seed.jl")
@everywhere include("CYCLOPS_v6_2a_PreNPostprocessModule.jl")
@everywhere include("CYCLOPS_v6_2a_CircularStats_U.jl")
@everywhere include("CYCLOPS_v6_2a_MultiCoreModule_Smooth.jl")



using CYCLOPS_v6_2a_AutoEncoderModule_multi
using CYCLOPS_v6_2a_Seed
using CYCLOPS_v6_2a_PreNPostprocessModule
using CYCLOPS_v6_2a_CircularStats_U
using CYCLOPS_v6_2a_MultiCoreModule_Smooth

# datadir = string("/circa/CYCLOPS/baboon")
# seeddir = string("/circa/CYCLOPS/baboon")

## Load datasets and seed gene list
* **Dataset**: Each row represents a gene, and each column represents a sample. 
  * The first row is the header **[1,:]**. 
  * The second column is gene symbol **[:,2]** 
  * The expression data strat from the fourth coulmn **[2:end, 4:end]**. 
* **Seed gene list**: Choose a good seed gene list is quite tricky. According to the suggestion from Dr. Anafi: The seed genes are supposed to reflect a "best guess" at the transcripts that will show circadian oscillations in the tissue being sorted. I do think using the seed list from the human data is a good way to go. 
  * Seed gene list is a two dimentions array with all the seed genes stored at the secondary column **[2:end, 2]**. 

In [2]:
# Load dataset
# cd(datadir)
data = readcsv("gtexExpr_lung.gct")

# Laod seed gene list
# cd(seeddir)
seed= readcsv("CircaGenes_baboon_edited.txt")
seed_list = seed[2:end,2]

cd(workdir)

## Set parameter
* **Frac_Var**: Set Number of Dimensions of SVD to maintain this fraction of variance
* **DFrac_Var**: Set Number of Dimensions of SVD to so that incremetal fraction of variance of var is at least this much
* **N_trials**: Number of random initial conditions to try for each optimization
* **MaxSeeds**: For cutrank = number of probes(genes) - Maxseeds
* **total_background_num**: Number of background runs for global background refrence statistics
* **n_cores**: Number of machine cores

In [3]:
Frac_Var = 0.85 
DFrac_Var = 0.03 
N_trials =20  
MaxSeeds = 10000
total_background_num=20
n_cores=5

5

## Subset datasets and set seeds

In [4]:
# Extract gene expression
expr = data[2:end, 4:end]
expr = Array{Float64}(expr) 

# Extract gene symbols of gene expression matrix
expr_symbol = data[2:end, 2]

# The number of probes(nprobes) can be used as the total genes input in CYCLOPS
# Because the script was written for microarray data at the first time
nprobes=size(expr)[1]
cutrank=nprobes-MaxSeeds
seed_MinMean=(sort(vec(mean(expr,2))))[cutrank] #mean of dim=2 (mean of rows/genes)

# Reseed the random number generator
srand(123456)

MersenneTwister(UInt32[0x0001e240], Base.dSFMT.DSFMT_state(Int32[-1659974861, 1073470649, 358281436, 1073351076, -836744907, 1073086183, 1497051007, 1072821109, 960204239, 1073516083  …  -761652241, 1073037584, -1988158346, 1073300352, -1977759271, 1347447899, 1840933752, -1237490475, 382, 0]), [1.70228, 1.465, 1.38226, 1.98167, 1.91602, 1.3244, 1.52647, 1.47618, 1.73525, 1.00613  …  1.24237, 1.69368, 1.8087, 1.73456, 1.81115, 1.76033, 1.63115, 1.0412, 1.42045, 1.1373], 382)

In [5]:
# Get gene symbol and expression data from seed genes
seed_symbol, seed_data = getseed(data,seed_list,.7,.14,seed_MinMean,.975)
seed_data_dispersion = dispersion!(seed_data)
outs, norm_seed_data = GetEigenGenes(seed_data_dispersion,Frac_Var,DFrac_Var,30)

(7, [0.0163033 -0.107071 … -0.400662 -0.0595415; 0.355214 -0.254532 … -0.134685 0.531912; … ; -0.425889 0.530591 … 0.46102 -0.0448329; -0.26226 -0.0556042 … -0.0998114 0.106407])

## CYCLOPS ordering

In [6]:
# Predict estimated phases
estimated_phaselist,bestnet,global_var_metrics=CYCLOPS_Order_multicore(outs,norm_seed_data,N_trials)

# Smoothing
global_smooth_metrics = smoothness_measures(seed_data_dispersion, norm_seed_data, estimated_phaselist)
pvals = multicore_backgroundstatistics_global_eigen(seed_data_dispersion, outs, N_trials, total_background_num, global_var_metrics)

	From worker 6:	15000 of max 15000 stochastic epochs
	From worker 5:	15000 of max 15000 stochastic epochs
	From worker 4:	15000 of max 15000 stochastic epochs
	From worker 2:	15000 of max 15000 stochastic epochs


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[6], in expression starting on line 2


	From worker 3:	15000 of max 15000 stochastic epochs
	From worker 5:	580 of max 5000.0 non-stochastic epochs
	From worker 5:	108 of max 5000 bold epochs
	From worker 2:	869 of max 5000.0 non-stochastic epochs
	From worker 5:	0.4515430182455311715000 of max 15000 stochastic epochs
	From worker 2:	108 of max 5000 bold epochs
	From worker 3:	966 of max 5000.0 non-stochastic epochs
	From worker 4:	1065 of max 5000.0 non-stochastic epochs
	From worker 3:	111 of max 5000 bold epochs
	From worker 6:	1167 of max 5000.0 non-stochastic epochs
	From worker 4:	111 of max 5000 bold epochs
	From worker 2:	0.459296486331197815000 of max 15000 stochastic epochs
	From worker 6:	110 of max 5000 bold epochs
	From worker 5:	355 of max 5000.0 non-stochastic epochs
	From worker 3:	0.4552087928667420415000 of max 15000 stochastic epochs
	From worker 5:	109 of max 5000 bold epochs
	From worker 4:	0.458478983748919415000 of max 15000 stochastic epochs
	From worker 6:	0.4552852866820112415000 of max 15000 stoch

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[6], in expression starting on line 6


	From worker 6:	0.4568712676915095415000 of max 15000 stochastic epochs
	From worker 3:	0.4572424572256413315000 of max 15000 stochastic epochs
	From worker 6:	298 of max 5000.0 non-stochastic epochs
	From worker 6:	105 of max 5000 bold epochs
	From worker 2:	424 of max 5000.0 non-stochastic epochs
	From worker 2:	104 of max 5000 bold epochs
	From worker 5:	574 of max 5000.0 non-stochastic epochs
	From worker 3:	632 of max 5000.0 non-stochastic epochs
	From worker 5:	108 of max 5000 bold epochs
	From worker 6:	0.4553498932788067615000 of max 15000 stochastic epochs
	From worker 3:	108 of max 5000 bold epochs
	From worker 2:	0.453701808563730415000 of max 15000 stochastic epochs
	From worker 5:	0.4590594669525235315000 of max 15000 stochastic epochs
	From worker 3:	0.4592759968042824415000 of max 15000 stochastic epochs
	From worker 4:	1169 of max 5000.0 non-stochastic epochs
	From worker 5:	306 of max 5000.0 non-stochastic epochs
	From worker 2:	481 of max 5000.0 non-stochastic epochs


...

	From worker 3:	0.4558162248321372415000 of max 15000 stochastic epochs
	From worker 5:	0.455449897475928815000 of max 15000 stochastic epochs
	From worker 2:	620 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 4:	0.4588313785570879415000 of max 15000 stochastic epochs
	From worker 2:	0.458334435130032415000 of max 15000 stochastic epochs
	From worker 5:	511 of max 5000.0 non-stochastic epochs
	From worker 4:	368 of max 5000.0 non-stochastic epochs
	From worker 3:	632 of max 5000.0 non-stochastic epochs
	From worker 5:	105 of max 5000 bold epochs
	From worker 6:	931 of max 5000.0 non-stochastic epochs
	From worker 4:	105 of max 5000 bold epochs
	From worker 3:	109 of max 5000 bold epochs
	From worker 6:	40 of max 5000 bold epochs
	From worker 5:	0.457728311706495915000 of max 15000 stochastic epochs
	From worker 4:	0.4583133835934856715000 of max 15000 stochastic epochs
	From worker 3:	0.45775388035726315000 of max 15000 stochastic epochs
	F

	From worker 5:	0.46107091607114915000 of max 15000 stochastic epochs
	From worker 3:	560 of max 5000.0 non-stochastic epochs
	From worker 3:	107 of max 5000 bold epochs
	From worker 3:	0.454630859841130815000 of max 15000 stochastic epochs
	From worker 2:	892 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 5:	933 of max 5000.0 non-stochastic epochs
	From worker 2:	0.455648017494213315000 of max 15000 stochastic epochs
	From worker 5:	112 of max 5000 bold epochs
	From worker 3:	701 of max 5000.0 non-stochastic epochs
	From worker 5:	0.459918156084203515000 of max 15000 stochastic epochs
	From worker 3:	108 of max 5000 bold epochs
	From worker 4:	1747 of max 5000.0 non-stochastic epochs
	From worker 4:	107 of max 5000 bold epochs
	From worker 2:	572 of max 5000.0 non-stochastic epochs
	From worker 3:	0.4582338945584304415000 of max 15000 stochastic epochs
	From worker 2:	107 of max 5000 bold epochs
	From worker 4:	0.457941551660184215000 of m

	From worker 5:	491 of max 5000.0 non-stochastic epochs
	From worker 5:	107 of max 5000 bold epochs
	From worker 6:	0.456518431699558915000 of max 15000 stochastic epochs
	From worker 2:	574 of max 5000.0 non-stochastic epochs
	From worker 2:	107 of max 5000 bold epochs
	From worker 5:	0.455060334253058515000 of max 15000 stochastic epochs
	From worker 2:	0.455134204049749715000 of max 15000 stochastic epochs
	From worker 4:	654 of max 5000.0 non-stochastic epochs
	From worker 4:	107 of max 5000 bold epochs
	From worker 6:	814 of max 5000.0 non-stochastic epochs
	From worker 4:	0.4603455206233371715000 of max 15000 stochastic epochs
	From worker 6:	108 of max 5000 bold epochs
	From worker 5:	855 of max 5000.0 non-stochastic epochs
	From worker 5:	109 of max 5000 bold epochs
	From worker 6:	0.4601753433212091315000 of max 15000 stochastic epochs
	From worker 4:	388 of max 5000.0 non-stochastic epochs
	From worker 4:	104 of max 5000 bold epochs
	From worker 4:	0.454640540636401215000 of 

	From worker 6:	105 of max 5000 bold epochs
	From worker 4:	0.4540237985892665615000 of max 15000 stochastic epochs
	From worker 6:	0.455708009930728115000 of max 15000 stochastic epochs
	From worker 3:	954 of max 5000.0 non-stochastic epochs
	From worker 3:	105 of max 5000 bold epochs
	From worker 3:	0.4535692935767123315000 of max 15000 stochastic epochs
	From worker 2:	3598 of max 5000.0 non-stochastic epochs
	From worker 2:	105 of max 5000 bold epochs
	From worker 6:	588 of max 5000.0 non-stochastic epochs
	From worker 6:	105 of max 5000 bold epochs
	From worker 2:	0.456889495518842815000 of max 15000 stochastic epochs
	From worker 6:	0.4562615817852978515000 of max 15000 stochastic epochs
	From worker 4:	1031 of max 5000.0 non-stochastic epochs
	From worker 4:	108 of max 5000 bold epochs
	From worker 3:	813 of max 5000.0 non-stochastic epochs
	From worker 3:	105 of max 5000 bold epochs
	From worker 3:	0.4555748038497806615000 of max 15000 stochastic epochs
	From worker 6:	1259 of 

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

## Cosinor statistics
**cosinor table:**

1.  ""            
2. "Description" (Gene Symbol)
3. "Name"       (Gene ID)
4. "pval"       
5. "bon_pval"   
6. "phase"      
7. "amp"        
8. "fitmean"    
9. "mean"       
10. "rsq"        
11. "ptr"

In [12]:
# Import Filter_cosinor_Output function for filtering bon_pval, ptr, rsq in the cosinor result
function Filter_Cosinor_Output(cosdata::Array{Any,2},pval,ptr,rsq) 
    significant_data=cosdata[append!([1],1 + findin(cosdata[2:end,5] .< pval,true)),:];
    phys_sig_data=significant_data[append!([1],1 + findin(significant_data[2:end,11] .> ptr,true)),:];
    strong_data=phys_sig_data[append!([1],1 + findin(phys_sig_data[2:end,10].> rsq,true)),:];
    strong_data
end

Filter_Cosinor_Output (generic function with 1 method)

In [13]:
estimated_phaselist = mod.(estimated_phaselist, 2*pi)
cosinor = Compile_MultiCore_Cosinor_Statistics(data, estimated_phaselist, 4, 24)
# Filter bon_pval<0.05, ptr>1.66, rsq>0
sig_cosinor = Filter_Cosinor_Output(cosinor, 0.05, 1.66, 0)

elapsed time: 4.146089117 seconds


5009×11 Array{Any,2}:
      ""  "Description"    …       "mean"   "rsq"     "ptr" 
    49    "LINC00115"             7.48525  0.191046  2.07094
    58    "NOC2L"                58.1188   0.552648  1.9374 
    60    "PLEKHN1"               2.56066  0.164004  1.93586
    63    "HES4"                 41.2178   0.296457  3.55523
    79    "TNFRSF18"       …      7.45445  0.313633  3.44424
    82    "B3GALT6"              15.2143   0.689985  3.97157
    83    "C1QTNF12"              3.19392  0.328829  3.6257 
    86    "LINC01786"             3.47622  0.273633  3.02308
    87    "SCNN1D"                6.12297  0.187137  2.36065
    90    "PUSL1"          …     16.5357   0.415313  1.78924
    94    "CPTP"                 24.031    0.463201  2.16727
    99    "AURKAIP1"             75.5757   0.462633  1.77921
     ⋮                     ⋱                         ⋮      
 55532    "SNORA70"             106.508    0.39109   2.03618
 55533    "DNASE1L1"             20.6142   0.430715  1.84927
 5

## Align all genes with Ebox(Par-BZip) phase
Ebox genes: DBP, HLF, TEF

According to Dr. Anafi, Ebox genes in mouse lung peak on average at CT 11.5

In [39]:
eboxgenes = ["DBP", "HLF", "TEF"]
eboxcosinor = cosinor[findin(cosinor[:,2], eboxgenes), :]

# Criteria(? not sure what is it for
# 4=pval,  9=mean, 11=ptr
# Unable to meet the original mean cutoff(100)
# Because the code is designed for microarray datasets, the mean of expression value are belongs to array expression values
# However, here we are try to analyse GTEx gene expression profile which expression values are TPM, so mean is set to 10 
criteria = ((eboxcosinor[:,4].<0.05) &(eboxcosinor[:,9].>10) & (eboxcosinor[:,11].>1.25)) 
eboxcosinor_fit = eboxcosinor[findin(criteria, true), :]

# Extract phases of Ebox genes
eboxphases = Array{Float64}(eboxcosinor[:,6])
eboxphases_mean = Circular_Mean(eboxphases)
estimated_phaselist_adj = mod.(estimated_phaselist .- eboxphases_mean + (pi), 2*pi)

# Count the number of genes which phase>pi or phase<pi
Nday=length(findin((estimated_phaselist_adj .>pi), true))
Nnight=length(findin((estimated_phaselist_adj .<pi), true))

# Adjust the numebr of the genes belongs to day and night if the number of night gene > the number of day gene
# e.g. 3/2*pi goes to 1/2*pi
if (Nday < Nnight)
    estimated_phaselist_adj = mod.(2*pi - estimated_phaselist_adj, 2*pi)
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] &(::BitArray{1}, ::BitArray{1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/execute_request.jl:193
 [5] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/pinjouwu9325/.julia/v0.6/Compat/src/Compat.jl:125
 [6] eventloop(::ZMQ.Socket) at /home/pinjouwu9325/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##13#16)() at ./task.jl:335
while loading In[39], in expression starting on line 9


578-element Array{Float64,1}:
 5.69501 
 3.17662 
 1.20624 
 3.65001 
 3.67828 
 0.914849
 1.56923 
 4.50254 
 4.70475 
 3.95195 
 2.89565 
 2.32352 
 2.0593  
 ⋮       
 1.30236 
 1.57332 
 1.47427 
 3.60191 
 4.38106 
 3.30967 
 3.42444 
 4.62035 
 2.15787 
 1.0901  
 3.30025 
 3.8368  

## Cosinor statistics after Ebox phase alignment
The input has changed to the adjusted phase after Ebox alignment

In [41]:
cosinor_adj = Compile_MultiCore_Cosinor_Statistics(data, estimated_phaselist_adj, 4, 24)
# Filter bon_pval<0.05, ptr>1.66, rsq>0
sig_cosinor_adj = Filter_Cosinor_Output(cosinor_adj, 0.05, 1.66, 0)

elapsed time: 3.687253496 seconds


5023×11 Array{Any,2}:
      ""  "Description"    …       "mean"   "rsq"     "ptr" 
    49    "LINC00115"             7.48525  0.191046  2.07094
    58    "NOC2L"                58.1188   0.552648  1.9374 
    60    "PLEKHN1"               2.56066  0.164004  1.93586
    63    "HES4"                 41.2178   0.296457  3.55523
    79    "TNFRSF18"       …      7.45445  0.313633  3.44424
    82    "B3GALT6"              15.2143   0.689985  3.97157
    83    "C1QTNF12"              3.19392  0.328829  3.6257 
    86    "LINC01786"             3.47622  0.273633  3.02308
    87    "SCNN1D"                6.12297  0.187137  2.36065
    90    "PUSL1"          …     16.5357   0.415313  1.78924
    94    "CPTP"                 24.031    0.463201  2.16727
    99    "AURKAIP1"             75.5757   0.462633  1.77921
     ⋮                     ⋱                         ⋮      
 55533    "DNASE1L1"             20.6142   0.430715  1.84927
 55535    "CH17-340M24.3"        17.9342   0.20892   2.19652
 5

## Save output file

In [63]:
# Set output directory
outdir = string(basedir, "/circa/CYCLOPS/baboon")
cd(outdir)
writecsv("Sig_Cosinor_lung.csv", sig_cosinor_adj)

In [64]:
# Output estimated phases
colnames = ["SAMPID" "Estimated_Phase"]
samplephase = hcat(data[1,4:end], estimated_phaselist_adj)
output = vcat(colnames, samplephase)
writecsv("GTEx_estimatedPhase_lung.csv", output)